### PostInstalls
Do these first by navigating to [POSTINSTALLS](http://ec2-54-210-238-172.compute-1.amazonaws.com/notebooks/Notebooks/PostInstall/PostInstalls.ipynb)

Then come on back! and continue with the next cell in this notebook
##### Author: dana
##### email: dana.e.moore@leidos.com

In [1]:
#import packages
import pandas as pd 
import numpy as np
import nltk
import re

import hdbscan
import umap
import contractions

from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer

from flair.embeddings import TransformerDocumentEmbeddings

import boto3
import logging
import uuid

# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import time
from datetime import date, datetime 
import datetime
from datetime import timedelta

## LOCAL 
CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
from APIutils import *

def complete(state="complete"):
#     print(f"Now: {datetime.datetime.now()}")
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state="imports done")


Cell imports done @ 2023-01-25 16:13:52


In [2]:
# # create a new dataframe from existing data (if available). If not then skip this step
# # get an inventory of the potential CSV files:
# csvfiles = [f for f in listdir(CSVPath) if isfile(join(CSVPath, f))]
# csvfiles.sort()

# csvfiles = [x for x in csvfiles if (os.path.splitext(x)[1][1:]).lower() == "csv"]
# complete(state='csvfiles done')

In [6]:
# csvsWD = widgets.Select(
#     options=csvfiles,
#     value=csvfiles[0],
#     description='Choose CSV:',
#     disabled=False,
#     button_style='success', # 'success', 'info', 'warning', 'danger' or ''
#         layout=Layout(width='50%', height='256px')
# )
# def on_change(change):
#     if change['type'] == 'change' and change['name'] == 'value':
# #         print ("changed to %s" % change['new'])
#         selection = change['new']

# csvsWD.observe(on_change)
# display(csvsWD)

jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [7]:
# csvfilename = osp.join (CSVPath,csvsWD.value)
# csvfilename
# iWD.value
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnf='semanticSimArticlesTestingDF.json' 


In [10]:
# df = pd.read_csv(csvfilename)
# # print(df.head())
# df
pd.set_option("max_colwidth", 30)
pd.set_option("expand_frame_repr", True)
jsnfile = osp.join (JSONPath,jsnf)
df = pd.read_json(jsnfile)
df

,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg
0,https://datainnovation.org...,economist created visualiz...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.429117,0.425113,0.410665,0.408926,0.408615,0.416487
1,https://venturebeat.com/ai...,among organization buildin...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.502411,0.485434,0.473214,0.469388,0.489188
2,https://venturebeat.com/ai...,watch demand session today...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.534699,0.533749,0.485653,0.485376,0.470144,0.501924
3,https://venturebeat.com/ai...,generative going completel...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.530356,0.528229,0.519292,0.488997,0.482039,0.509782
4,https://www.businessinside...,personalized cancer vaccin...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.438377,0.427320,0.422924,0.419798,0.414401,0.424564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,https://www.foxbusiness.co...,struggling current economy...,0.5382,0.5022,0.4537,0.4530,0.4166,0.472740,0.421236,0.419552,0.417580,0.389859,0.387455,0.407136
112,https://www.foxbusiness.co...,latest take varney host st...,0.5482,0.4926,0.4814,0.4076,0.4005,0.466060,0.443807,0.422615,0.377209,0.352440,0.342977,0.387810
113,https://www.foxbusiness.co...,famed serial killer john w...,0.2994,0.2344,0.2231,0.2163,0.2147,0.222125,0.277694,0.273635,0.253718,0.246481,0.240336,0.258373
114,https://www.foxbusiness.co...,housing market experiencin...,0.6052,0.5847,0.5802,0.5490,0.5490,0.573620,0.493047,0.464461,0.457469,0.449820,0.444688,0.454109


In [11]:
articleslist = df['article'].tolist()
articleslist = df.values.tolist()

articleslist

[['https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/',
  'economist created visualization showing politician twitter follower changed elon musk acquired platform visualization show percent change individual account party average october october according visualization republican gained around follower musk acquired twitter democrat lost follower take look',
  0.5627,
  0.5556,
  0.5153,
  0.5132,
  0.4939,
  0.52814,
  0.4291173518,
  0.42511266470000003,
  0.4106646776,
  0.40892553330000003,
  0.4086146653,
  0.4164869785],
 ['https://venturebeat.com/ai/stability-ai-looks-to-aws-cloud-to-power-the-next-generation-of-generative-ai/',
  'among organization building generative technology stability raised million funding october building generative model like stable diffusion requires significant amount computing power training inference aws invent conference week la vega stability formally announced chosen aws preferred cloud platform building g

In [16]:
# for each article URL in the dataframe:
#     read the summary
#     do BERTopic scoring for the article
#     add the top-n columns to the dataframe
#     re-serialise in JSON format
sum_list = []
for article in articleslist:
    summary = article[1]
    sum_list.append(summary)
#     print (f"Summary: {summary}", end="\n\n")
    

In [17]:
print(sum_list)

['economist created visualization showing politician twitter follower changed elon musk acquired platform visualization show percent change individual account party average october october according visualization republican gained around follower musk acquired twitter democrat lost follower take look', 'among organization building generative technology stability raised million funding october building generative model like stable diffusion requires significant amount computing power training inference aws invent conference week la vega stability formally announced chosen aws preferred cloud platform building generative tool watch herethe cloud generative hand handstability generative vendor relies public cloud help build foundation model stable diffusion us cloud generate image fasterbuilding stable diffusion exercise involves multiple step component', 'watch demand session today watch herealtman latest cautionary note probably responding last week beat column seems chatgpt horse fully

In [18]:
# sum_list=[]

# # for i in articles['Items']:
# #     sum_list.append({i['summary']})

# for article in articleslist:
#     summary = article[2]
#     sum_list.append(summary)    

df = pd.DataFrame(sum_list, columns =['text'])

# remove protocols
df.text = df.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)

# remove non-letters
df.text = df.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)

# convert to lowercase
df.text = df.text.apply(lambda x: ' '.join([w.lower() for w in x.split()]))

# expand contractions  
df.text = df.text.apply(lambda x: ' '.join([contractions.fix(w) for w in x.split()]))

# remove stopwords
stop_words = stopwords.words('english')
df.text = df.text.apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words]))

# remove short words
df.text = df.text.apply(lambda x: ' '.join([w.strip() for w in x.split() if len(w.strip()) >= 3]))

# lemmatize
df.text = df.text.apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(w) for w in x.split()]))

docs = df.text.to_list()

complete(state="dataframe data massage")


Cell dataframe data massage @ 2023-01-25 16:20:06


In [19]:
%%time
# create model

# # Load sentence transformer model
# sentence_model = SentenceTransformer("all-distilroberta-v1")

# # Create documents embeddings
# embeddings = sentence_model.encode(docs, show_progress_bar=True)
roberta = TransformerDocumentEmbeddings('roberta-large')

# Define UMAP model to reduce embeddings dimension
umap_model = umap.UMAP(n_neighbors=9,
                       n_components=5,
                       min_dist=0.0,
                       metric='cosine',
                       low_memory=False)

# Define HDBSCAN model to perform documents clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=3,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True)

# Create BERTopic model
topic_model = BERTopic(embedding_model=roberta,
#                        nr_topics=1,
                       top_n_words=5,
                       n_gram_range=(1,3), 
                       calculate_probabilities=True,
                       umap_model= umap_model,
                       hdbscan_model=hdbscan_model,
                       verbose=True)


complete(state="models")


Cell models @ 2023-01-25 16:20:30
CPU times: user 4.43 s, sys: 2.95 s, total: 7.38 s
Wall time: 7.21 s


In [20]:
%%time
# Train model, extract topics and probabilities
topics, probabilities = topic_model.fit_transform(docs)

complete(state="topics, probabilities done")

116it [00:04, 25.51it/s]
2023-01-25 16:20:42,466 - BERTopic - Transformed documents to Embeddings
2023-01-25 16:20:51,225 - BERTopic - Reduced dimensionality
2023-01-25 16:20:51,253 - BERTopic - Clustered reduced embeddings



Cell topics, probabilities done @ 2023-01-25 16:20:52
CPU times: user 15.3 s, sys: 198 ms, total: 15.5 s
Wall time: 14.9 s


In [29]:
# info = topic_model.get_topic_info()
topic_model.get_topic_info()
# print (f"topic model info : {info};\n# topics :{len(info)}", end="\n\n")
# for topic in range(0,len(info)):
#     print (f" Topic (topic) : \n\t{topic_model.get_topic(topic)}")


,Topic,Count,Name
0,-1,4,-1_build_hisense_aws_building
1,0,18,0_cooky_sale_way constitut...
2,1,18,1_said_republican_image_house
3,2,16,2_billion_synthetic_theft_...
4,3,15,3_digital_innovation_techn...
5,4,10,4_developer_web_web develo...
6,5,7,5_space_nasa_spacex_ozone
7,6,7,6_azure_iot_aws_contract
8,7,6,7_vehicle_waste_nuclear_china
9,8,5,8_vaccine_cancer_group_shi...


In [23]:
# get docs from topic

df = pd.DataFrame({'topic': topics, 'document': docs})
docs_in_topic = df[df.topic == 1]
docs_in_topic


,topic,document
0,1,economist created visualiz...
8,1,suggestion fcc could adopt...
17,1,vega nevada despite divide...
24,1,choose plan arrowright spe...
25,1,push satellite communicati...
46,1,paycheck protection progra...
48,1,security force regain cont...
49,1,historic house speaker sho...
51,1,seattle school suing tech ...
52,1,republican elite backed tr...


In [26]:
topic_words = topic_model.get_topic(1)

print(topic_words)

[('said', 0.01959022792058907), ('republican', 0.013655351159498738), ('image', 0.013011737746390922), ('house', 0.012413955599544307), ('medium', 0.011381536426707669)]


In [27]:
topic_model.visualize_barchart(top_n_topics=8)

In [30]:
topic_model.visualize_topics()

In [31]:
tpcs = []
for t in info["Topic"][0:int(len(info))]: 
    tpcarray = topic_model.get_topic(t)
    for elt in tpcarray:
        tpcs.append(elt[0])
tpcs = list (set(tpcs))
tpcs.sort()
print (tpcs)

['adobe', 'award', 'aws', 'azure', 'billion', 'browser', 'build', 'building', 'cancer', 'chatgpt', 'china', 'clown', 'company', 'constitutes sale', 'contract', 'cooky', 'developer', 'digital', 'encrochat', 'endangered', 'endangered specie', 'generative', 'government', 'group', 'hisense', 'house', 'image', 'information', 'innovation', 'invasive', 'iot', 'loss', 'medium', 'nasa', 'network', 'new', 'nuclear', 'ozone', 'painting', 'personal', 'personal information', 'plastic', 'republican', 'rocket', 'said', 'sale', 'security', 'shingle', 'space', 'spacex', 'specie', 'stock', 'store', 'synthetic', 'technology', 'theft', 'transformation', 'vaccine', 'vehicle', 'waste', 'way constitutes', 'way constitutes sale', 'web', 'web developer']


In [36]:
# topic_model.probabilities_[7]
# topic_model.visualize_distribution(topic_model.probabilities_[5], min_probability=0.005)

In [38]:
jsonfile = join(JSONPath, "semanticSimArticlesTesting.json")
# read this in as an array

with open(jsonfile, "r") as infile:
    element = json.load(infile)
for key in element.keys():
    print (f"key: {key}")
    print (f"\t\tSummary: {element[key]['summary']}")

key: https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/
		Summary: The Economist has created a visualization showing how politicians’ Twitter followers changed after Elon Musk acquired the platform.
The visualization shows the percent change for both individual accounts and party averages from October 24 to October 30.
According to the visualization, Republicans gained around 470,000 followers after Musk acquired Twitter while Democrats lost 420,000 followers.
Take a look.
key: https://venturebeat.com/ai/stability-ai-looks-to-aws-cloud-to-power-the-next-generation-of-generative-ai/
		Summary: Among the organizations building generative AI technologies is Stability AI, which raised $101 million in funding in October.
Building a generative AI model like Stable Diffusion requires a significant amount of computing power both for training and for inference.
At the AWS re:Invent conference this week in Las Vegas, Stability AI formally announced that i

In [39]:
test_query = element['https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/']['summary']

# Find topics
num_of_topics = 2
similar_topics, similarity = topic_model.find_topics(test_query, top_n=num_of_topics)

# Print results
print(f'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,4)}')

The top 2 similar topics are [7, 9], and the similarities are [0.9988 0.9988]


In [40]:
# Print the top keywords for the top similar topics
for i in range(num_of_topics):
    print(f'The top keywords for topic {similar_topics[i]} are:')
    print(topic_model.get_topic(similar_topics[i]))

The top keywords for topic 7 are:
[('vehicle', 0.0321154030764886), ('waste', 0.028993121723851923), ('nuclear', 0.028993121723851923), ('china', 0.026927958132339416), ('plastic', 0.024935639312814135)]
The top keywords for topic 9 are:
[('painting', 0.030278990594131448), ('encrochat', 0.030278990594131448), ('network', 0.02785519654593399), ('clown', 0.025144796402737347), ('new', 0.024531338166156744)]


In [ ]:
# Heat Maps, Rankings, Barcharts